In [1]:
!wget -c https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K/resolve/main/llava_instruct_150k.json

--2024-09-27 20:01:54--  https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K/resolve/main/llava_instruct_150k.json
Resolving huggingface.co (huggingface.co)... 18.244.202.118, 18.244.202.68, 18.244.202.60, ...
Connecting to huggingface.co (huggingface.co)|18.244.202.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/4d/41/4d41ea1e2709f0e68e9e361e4218192b9620c5a3f2cb8055bc625942b6cd3039/6b68bc5ca2bfd8a71119af0e8454929668ccda6a334955ccc95d114fc8d082fa?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27llava_instruct_150k.json%3B+filename%3D%22llava_instruct_150k.json%22%3B&response-content-type=application%2Fjson&Expires=1727726515&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNzcyNjUxNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy80ZC80MS80ZDQxZWExZTI3MDlmMGU2OGU5ZTM2MWU0MjE4MTkyYjk2MjBjNWEzZjJjYjgwNTViYzYyNTk0MmI2Y2QzMDM5LzZiNjhiYzVjYTJiZmQ4YTcxMTE5Y

In [2]:
!wget -c http://images.cocodataset.org/zips/train2017.zip
!unzip -q train2017.zip -d coco/

--2024-09-27 20:01:55--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.201.1, 52.217.108.180, 52.217.124.113, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.201.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip       100%[===================>]  18.01G  94.9MB/s    in 3m 13s  

2024-09-27 20:05:09 (95.4 MB/s) - ‘train2017.zip’ saved [19336861798/19336861798]



In [3]:
!pip install -q accelerate transformers trl git+https://github.com/huggingface/peft.git datasets bitsandbytes einops wandb git+https://github.com/openai/CLIP.git ftfy regex tqdm

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import json
import os
import torch
import numpy as np
from PIL import Image
import clip
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig
from datasets import load_dataset

# 1. Load the dataset and parse image paths
dataset = load_dataset('json', data_files='llava_instruct_150k.json', split='train')

# 2. Initialize CLIP model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
clip_model, preprocess = clip.load("ViT-B/32", device=device)

# 3. Get image embeddings
def process_and_save_embeddings(dataset, embedding_file='image_embeddings.npy'):
    if os.path.exists(embedding_file):
        print("Loading existing embeddings...")
        return np.load(embedding_file)

    print("Computing and saving embeddings...")
    image_embeddings = []
    for entry in dataset:
        image_path = os.path.join('coco/train2017', entry['image'])
        if os.path.exists(image_path):
            img = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
            with torch.no_grad():
                embedding = clip_model.encode_image(img)
                image_embeddings.append(embedding.cpu().numpy())

    image_embeddings = np.array(image_embeddings)
    np.save(embedding_file, image_embeddings)
    return image_embeddings

image_embeddings = process_and_save_embeddings(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 150MiB/s]


Computing and saving embeddings...


In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
!mv /content/image_embeddings.npy /content/drive/MyDrive/ERA\ v2/multimodal-system